In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

### Conv2d

In [3]:
ch_in = 2
ch_out = 100
model = nn.Conv2d(ch_in, ch_out, 7)
#
x = torch.rand(1, ch_in, 7, 7)
y = model(x)
y.shape

torch.Size([1, 100, 1, 1])

In [4]:
for p in model.parameters():
    print(p.shape)

torch.Size([100, 2, 7, 7])
torch.Size([100])


### Group Convolution

In [5]:
import sys
sys.path.append("./../..")
from misc.utils import count_parameters

In [6]:
ch_in = 4
ch_out = 8
model = nn.Conv2d(ch_in, ch_out, 7, groups=4)
#
x = torch.rand(1, ch_in, 7, 7)
y = model(x)
for p in model.parameters():
    print(p.shape)

torch.Size([8, 1, 7, 7])
torch.Size([8])


In [7]:
model = nn.Sequential(
    nn.Conv2d(3, 128, 3, groups=1, stride=1),
    nn.Conv2d(128, 256, 3, groups=128, stride=2),
    nn.Conv2d(256, 512, 3, groups=256, stride=1),
    nn.Conv2d(512, 1024, 3, groups=512, stride=2),
)
count_parameters(model)

21504

In [8]:
x = torch.rand(1, 3, 32, 32)
y = model(x)
y.shape

torch.Size([1, 1024, 5, 5])

In [9]:
n_caps = 32
d_caps = 16
n_groups = n_caps
d_out = n_caps * d_caps
#
model = nn.Sequential(
    nn.Conv2d(3, 256, kernel_size=3, groups=1, stride=2, padding=0),
    nn.Conv2d(256, 512, kernel_size=3, groups=n_groups, stride=2, padding=0),
    nn.Conv2d(512, d_out, kernel_size=7, groups=n_groups, stride=1, padding="valid"),
)
print(count_parameters(model))
#
x = torch.rand(128, 3, 32, 32)
y = model(x)
y.shape

446464


torch.Size([128, 512, 1, 1])

In [10]:
model = nn.Sequential(
    nn.Conv2d(16, 16, kernel_size=3, groups=16, stride=1, padding="valid", bias=False),
    nn.Conv2d(16, 16, kernel_size=3, groups=16, stride=1, padding="valid", bias=False),
    nn.Conv2d(16, 16, kernel_size=3, groups=16, stride=1, padding="valid", bias=False),
)
print(count_parameters(model))

432


In [11]:
x = torch.ones(1, 16, 7, 7)
x[:, 1:, :, :] = 0
y = model(x)
y.reshape(1, 16)

tensor([[0.0862, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]],
       grad_fn=<ReshapeAliasBackward0>)

In [12]:
y.shape

torch.Size([1, 16, 1, 1])

In [13]:
model = nn.Sequential(
    nn.Conv2d(3, 512, kernel_size=3, groups=1, stride=1, padding="valid", bias=False),
    nn.Conv2d(512, 1024, kernel_size=3, groups=256, stride=1, padding="valid", bias=False),
    nn.Conv2d(1024, 2048, kernel_size=3, groups=256, stride=1, padding="valid", bias=False),
)
print(count_parameters(model))

105984


In [14]:
2048 / 256

8.0

In [15]:
model = nn.Sequential(
            nn.Conv2d(3, 512, kernel_size=3, groups=1, stride=2, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv2d(512, 1024, kernel_size=3, groups=256, stride=2, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv2d(1024, 2048, kernel_size=3, groups=256, stride=2, padding=0, bias=False),
            nn.ReLU(),
            nn.Conv2d(2048, 2048, kernel_size=3, groups=256, stride=1, padding=0, bias=False),
)
print(count_parameters(model))

253440


In [42]:
model = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, groups=1, stride=1, padding="same", bias=False),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 1024, kernel_size=3, groups=64, stride=1, padding="same", bias=False),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(1024, 1024, kernel_size=3, groups=64, stride=1, padding=0, bias=False),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(1024, 1024, kernel_size=3, groups=64, stride=1, padding=0, bias=False),

)
print(count_parameters(model))
model(torch.rand(1, 3, 32, 32)).shape

305856


torch.Size([1, 1024, 1, 1])

In [26]:
1024 / 64

16.0

In [27]:
model(torch.rand(1, 3, 32, 32)).shape

torch.Size([1, 1024, 20, 20])